# Содержание:

    1) Метод word2vec с классификаторами:
            1) Ridge Classifier
            2) Perceptron
            3) Passive Agressive
            4) BeurnolliNB
            5) MultinomialNB
            6) KNeighbors
            7) RandomForest
            8) NearestCentroid
            9) SGDClassifier
            10) LinearSVC
    2) Визуализация


In [2]:
%matplotlib inline
import os
from lxml import etree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pymorphy2
import gensim
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

## Загружаю данные:

In [2]:
sphere_6 = defaultdict(int)
genre_fi_7 = defaultdict(int)
type_8 = defaultdict(int)
topic_9 = defaultdict(int)

i = 0
f = open('./out.csv', 'r')
for line in f:
    i += 1
    attrs = line.split(';')
    sphere_6[attrs[6]] += 1
    genre_fi_7[attrs[7]] += 1
    type_8[attrs[8]] += 1
    topic_9[attrs[9]] += 1

In [82]:
# type_8

In [80]:
# cl.keys()

In [3]:
from Levenshtein import distance

In [4]:
f = open('./out.csv', 'r')
file_hash_all = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    file_name = os.path.basename(path)
    dir_name = os.path.dirname(path)
    full_dir = os.path.join('/Users/Valeriya/Desktop/CoursePaper/corpus', dir_name)
    files = os.listdir(full_dir)
    if file_name in files:
        file_hash_all[file_name] = class_name
    elif file_name + '.xml' in files:
        file_hash_all[file_name + '.xml'] = class_name
    elif file_name + '.xhtml' in files:
        file_hash_all[file_name + '.xhtml'] = class_name
    else:
        edit = 1000
        true_name = ''
        for file_in_dir in files:
            dist = distance(file_name, file_in_dir)
            if dist < edit:
                edit = dist
                true_name = file_in_dir
        file_hash_all[true_name] = class_name

In [4]:
len(file_hash_all)

14382

In [5]:
f = open('./out.csv', 'r')
cl = {'отчет':0,'поздравление':0, 'интервью':0, 'репортаж':0, 'хроника':0, 'объявление':0, 'комментарий':0,
      'совет':0, 'анонс':0, 'заметка':0, 'информационное сообщение':0, 'статья':0}
file_hash = {}
for file_name, class_name in file_hash_all.items():
    if class_name in cl:
        cl[class_name] +=1
        file_hash[file_name] = class_name

In [6]:
len(file_hash)

9847

In [7]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    pos = p.tag.POS
    if pos:
        if 'ADJ' in pos:
            pos = 'ADJ'
    else:
        pos = ''
    return '{}_{}'.format(
        p.normal_form,
        pos
    )

2017-05-19 09:04:59,384 INFO Loading dictionaries from /Users/Valeriya/anaconda/lib/python3.5/site-packages/pymorphy2_dicts/data
2017-05-19 09:04:59,482 INFO format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [8]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text, open_name

In [27]:
# sent = []
# for tree, class_text, open_name in read_txt('./corpus'):
#     all_p = tree.xpath('//p')
#     joined = '\n'.join([p.text for p in all_p if p.text is not None])
#     sent.append((joined, class_text))

In [288]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [9]:
texts = []
for tree, class_text, open_name in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    body = tree.xpath('//body')
    text = '\n'.join([elem.text for elem in body[0] if elem.text])
#     if class_text != 'поздравление':
#         class_text = 'другие'
    if text:
        texts.append((text, class_text, open_name))
    else:
        print(open_name)

/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451403.xhtml
/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451404.xhtml


In [30]:
# print(texts[0])

In [291]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [10]:
m = 'ruwikiruscorpora_0_300_20.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

2017-05-19 09:05:27,915 INFO loading projection weights from ruwikiruscorpora_0_300_20.bin.gz
2017-05-19 09:06:01,918 INFO loaded (392339, 300) matrix from ruwikiruscorpora_0_300_20.bin.gz


In [13]:
features = {}
# with open('formal_features_3.tsv', 'r') as f:
with open('formal_features_3.tsv', 'r') as f:
    for line in f:
        items = line.rstrip().split('\t')
        path = items[0]
        feats = items[1::]
        features[path] = feats

In [12]:
only_texts = [text[0] for text in texts]
classes = [text[1] for text in texts]
        
data = []
for i, (text, class_name, open_name) in enumerate(texts):
    text_vectors = []
    words = text.split()
    for word in words:
        norm_word = normalize_and_add_pos(word)
        if norm_word in model:
            text_vectors.append(model[norm_word])
    mean_vector = sum(text_vectors) / len(text_vectors)
    open_name = open_name.replace('/Users/Valeriya/Desktop/CoursePaper/', './')
    formal_features = features[open_name][::]
    n_excl = 0
    for item in text[0]:
        if item == '!':
            n_excl+=1

    n_ques = 0
    for item in text[0]:
        if item == '?':
            n_ques+=1
    formal_features.append(n_excl)
    formal_features.append(n_ques)
    
    formal_features = [float(x) for x in formal_features]
    formal_features = formal_features[:2:] + [x/formal_features[0] for x in formal_features[2::]]
    
    data.append(list(mean_vector)+formal_features)


# data = pd.DataFrame(texts_norm)
# data['class'] = classes

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.3)

In [33]:
len(X_test)

2954

In [34]:
min_max_scaler = MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

## Классификаторы

In [1]:
from sklearn.metrics import classification_report, confusion_matrix

clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
print(classification_report(y_test, y_pred1))
print(confusion_matrix(y_test, y_pred1))
clf2 = LogisticRegression()
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)
print(classification_report(y_test, y_pred2))
print(confusion_matrix(y_test, y_pred2))


NameError: name 'RandomForestClassifier' is not defined

In [36]:
forest = RandomForestClassifier(n_estimators= 100)

In [37]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [38]:
y_pred = forest.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6431956668923493